Método de la M grande, 

# Parte 1: obtenemos los datos

In [1]:
import numpy as np
import deep_inv_opt as io
import deep_inv_opt.plot as iop
import torch

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.max_open_warning'] = 0  # Let the plots flow!
%matplotlib inline

In [2]:
u_train = io.tensor(np.linspace(0.5, 50, 1024).reshape((-1, 1)))
u_train

tensor([[ 0.5000],
        [ 0.5484],
        [ 0.5968],
        ...,
        [49.9032],
        [49.9516],
        [50.0000]], dtype=torch.float64)

ahora generamos los x correspondientes del modelo real

In [3]:
class ExamplePLP(io.ParametricLP):
    # Generate an LP from a given feature vector u and weight vector w.
    def generate(self, u, w):
        c = [[torch.cos(w + u**2 / 2)],
             [torch.sin(w + u**2 / 2)]]

        A_ub = [[-1.0,  0.0],      # x1 >= 0
                [ 0.0, -1.0],      # x2 >= 0
                [ 1.0,  0.0],      # x1 <= 2*w
                [ .5*w, w]]  # (1+w)*x1 + 2*(1+w)*x2 <= u

        b_ub = [[ 0.0],
                [ 0.0],
                [ 4/u],
                [   u]]
        
        return c, A_ub, b_ub, None, None

In [4]:
plp_true = ExamplePLP(weights=[0.8])

# Generate training targets by solve the true PLP at each u value.
# x_train = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_train])
# torch.save(x_train, "x_train.pt")

In [5]:
x_train = torch.load("x_train.pt")
x_train

tensor([[6.3378e-06, 4.7761e-06],
        [6.4988e-06, 4.6438e-06],
        [6.1619e-06, 4.1501e-06],
        ...,
        [8.0112e-02, 1.4527e-05],
        [1.9709e-04, 6.2439e+01],
        [2.5363e-05, 5.3045e-05]], dtype=torch.float64)

# Parte 2: definimos la red y la entrenamos

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [7]:
learning_rate = 1e-3
batch_size = 8
epochs = 100

In [8]:
# definimos el dataset
class UDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data.clone().to(dtype=torch.float32) # nota: esta dando el warning porque estoy convirtiendo un tensor a otro, en ese caso es mejor usar clone()
        # si los datos de entrada no los voy a dar como un tensor, entonces hay que poner lo que he puesto: self.data = torch.tensor(data, dtype=torch.float32), self.targets = torch.tensor(targets, dtype=torch.float32)
        self.targets = targets.clone().to(dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Dataset con pares (u, x)
# u_data = [[1.0], [2.0], [3.0], [4.0]]
# x_targets = [[1.0, 1.5], [2.0, 2.5], [3.0, 3.5], [4.0, 4.5]]
dataset = UDataset(u_train, x_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [9]:
# definimos la red (hay que revisar la forma de la red y el por qué)
class ParametricLPNet(nn.Module):
    def __init__(self):
        super(ParametricLPNet, self).__init__()
        # Entrada de dimensión 1, salida 8 (2 para c, 4 para A, 2 para b)
        self.fc = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, 8)  # c (2), A (4), b (2)
        )

    def forward(self, u):
        output = self.fc(u)
        c = output[:, 0:2]      # Vector de costes
        A = output[:, 2:6].reshape(-1, 2, 2)  # Matriz A (2x2)
        b = output[:, 6:8]      # Vector de restricciones
        return c, A, b


In [10]:
# funcion que resuelve el problema de programacion lineal (por ahora nos la creemos pero hay que revisarla)
# hay que tener en cuenta que esta funcion debe preservar el grafo de computo para poder hacer backpropagation
def smooth_lp(c, A, b):
    # Inicializar x con gradientes habilitados
    x = torch.zeros(A.shape[1], requires_grad=True)

    optimizer = torch.optim.SGD([x], lr=learning_rate)

    for _ in range(100):
        optimizer.zero_grad()
        constraint_penalty = torch.sum(torch.relu(A @ x - b))
        objective = torch.dot(c, x) + 1000.0 * constraint_penalty
        objective.backward(retain_graph=True)
        optimizer.step()
    return x  # Sin detach()


In [11]:
# funcion de perdida
def my_loss(c, A, b, target):
    rs = smooth_lp(c, A, b)
    loss = torch.sum((rs - target) ** 2)
    return loss

def loss_fn(c, A, b, target):
    return torch.mean(torch.stack([my_loss(c[i], A[i], b[i], target[i]) for i in range(len(c))]))

In [12]:
# Crear la red neuronal
model = ParametricLPNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # elegir una de las dos

In [13]:
def train_loop(dataloader, model, loss_fn, optimizer):
    for batch, (u_batch, x_batch) in enumerate(dataloader):
        size = len(dataloader.dataset)

        c, A, b = model(u_batch)
        # Calcular la pérdida
        #loss = my_loss(c[0], A[0], b[0], x_batch[0])  # Usar el target correspondiente
        loss = loss_fn(c, A, b, x_batch)

        # Backpropagation y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 8 == 0: # cambiar este numero para que salga cada cierto numero de iteraciones
            loss, current = loss.item(), batch * batch_size + len(u_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [17]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size=len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #with torch.no_grad():
    for u_batch, x_batch in dataloader:
        c, A, b = model(u_batch)
        test_loss += loss_fn(c, A, b, x_batch).item()
        x_pred = torch.stack([smooth_lp(c[i], A[i], b[i]) for i in range(len(c))])
        correct += torch.sum(x_pred == x_batch).item() # cambiar esto, aqui poner la solucion del problema
        # y consideramos correcto si se acerca a la solucion en la distancia euclidea 
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
# Entrenamiento
epochs = 4 # poner mas
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(dataloader, model, loss_fn, optimizer)
    test_loop(dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 751.658081  [    8/ 1024]
loss: 406.782806  [   72/ 1024]
loss: 323.025909  [  136/ 1024]
loss: 706.061523  [  200/ 1024]
loss: 270.391846  [  264/ 1024]
loss: 1138.208984  [  328/ 1024]
loss: 972.936401  [  392/ 1024]
loss: 536.850708  [  456/ 1024]
loss: 161.913574  [  520/ 1024]
loss: 630.460999  [  584/ 1024]
loss: 448.160828  [  648/ 1024]
loss: 878.794006  [  712/ 1024]
loss: 392.461365  [  776/ 1024]
loss: 636.737366  [  840/ 1024]
loss: 305.131714  [  904/ 1024]
loss: 430.907837  [  968/ 1024]
Test Error: 
 Accuracy: 0.0%, Avg loss: 639.000893 

Epoch 2
-------------------------------
loss: 910.739624  [    8/ 1024]
loss: 598.135925  [   72/ 1024]
loss: 673.777466  [  136/ 1024]
loss: 1124.274902  [  200/ 1024]
loss: 793.530579  [  264/ 1024]
loss: 814.863525  [  328/ 1024]
loss: 74.730316  [  392/ 1024]
loss: 615.075745  [  456/ 1024]
loss: 531.544312  [  520/ 1024]
loss: 398.201904  [  584/ 1024]
loss: 197.492126  [  648/ 1024]
lo

In [ ]:
torch.save(model.state_dict(), "model_weights.pth")

In [ ]:
c,A,b=model(torch.tensor([[1.0]]))
smooth_lp(c[0], A[0], b[0])

tensor([ 0.1192, -0.0461], requires_grad=True)

In [ ]:
en el dataloader hay alguna manera de añadirle un conjunto de validacion?

Object `validacion` not found.


In [ ]:
u_test = torch.tensor(np.linspace(0.1, 4, 16).reshape((-1, 1)), dtype=torch.float64)
u_test

tensor([[0.1000],
        [0.3600],
        [0.6200],
        [0.8800],
        [1.1400],
        [1.4000],
        [1.6600],
        [1.9200],
        [2.1800],
        [2.4400],
        [2.7000],
        [2.9600],
        [3.2200],
        [3.4800],
        [3.7400],
        [4.0000]], dtype=torch.float64)

ahora probamos como funciona para un x nuevo

In [ ]:
x_test = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_test])
x_test

tensor([[5.5031e-06, 5.2914e-06],
        [5.8791e-06, 5.0125e-06],
        [6.2946e-06, 4.1114e-06],
        [1.5288e-05, 6.1703e-06],
        [6.3201e-05, 7.6848e-06],
        [2.8571e+00, 7.7249e-06],
        [2.4096e+00, 8.3817e-06],
        [2.0833e+00, 1.4402e-05],
        [1.8349e+00, 1.8074e+00],
        [1.6393e+00, 2.2303e+00],
        [6.9978e-05, 3.3749e+00],
        [1.2749e-05, 3.7000e+00],
        [6.8515e-06, 4.0250e+00],
        [1.1909e-05, 1.8498e-05],
        [2.2888e-04, 1.3794e-05],
        [9.9999e-01, 1.9565e-05]], dtype=torch.float64)

In [ ]:
model.eval()

ParametricLPNet(
  (fc): Sequential(
    (0): Linear(in_features=1, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
  )
)

In [ ]:
u = torch.tensor([[0.25]], dtype=torch.float64)
x = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u])
u,x

(tensor([[0.2500]], dtype=torch.float64),
 tensor([[5.6596e-06, 5.1630e-06]], dtype=torch.float64))

In [ ]:
model(torch.tensor([[0.25]]))

(tensor([[ 0.1622, -0.2429]], grad_fn=<SliceBackward0>),
 tensor([[[-0.4103,  0.0860],
          [ 0.0938,  0.0168]]], grad_fn=<ViewBackward0>),
 tensor([[0.1193, 0.0358]], grad_fn=<SliceBackward0>))

In [ ]:
c, A, b = model(torch.tensor([[0.25]]))
x_pred = smooth_lp(c[0], A[0], b[0])

In [ ]:
x, x_pred, torch.norm(x - x_pred)

(tensor([[5.6596e-06, 5.1630e-06]], dtype=torch.float64),
 tensor([-0.0162,  0.0243], requires_grad=True),
 tensor(0.0292, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>))

In [ ]:
# se parecen bastante, ahora el siguiente paso es hacer la validacion, ya lo dejo para mañana